In [2]:
import itertools
import matplotlib.pyplot as plt
import pickle 
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
from QuSO_utils import *

In [54]:
# Environmental Parameters
T_env = 293       # Ambient temperature (K)
R_env = 0.01      # Convection resistance to ambient (K/W)

# Heat Flows (in Watts)
# Positive values indicate heat generation; negative values indicate cooling.
Q_1 = 2000    
Q_2 = 4000    
Q_3 = -200    
Q_4 = -2000   

# Inter-node Thermal Resistances (in K/W)
# These values lump together conduction and convection effects.
R_12 = 0.005
R_13 = 0.006
R_14 = 0.006
R_23 = 0.007
R_24 = 0.007
R_34 = 0.008

R_dict = {
    (0, 1): R_12,
    (0, 2): R_13,
    (0, 3): R_14,
    (1, 2): R_23,
    (1, 3): R_24,
    (2, 3): R_34
}

connections = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

conductance_coeffs = [1/R_12, 1/R_13, 1/R_14, 1/R_23, 1/R_24, 1/R_34, 1/(2*R_env), 0]
C_l = np.sum(conductance_coeffs)**(-1/2)
conductance_coeffs_amps = np.sqrt(conductance_coeffs)*C_l

B = np.array([Q_1, Q_2, Q_3, Q_4])
C_B = np.sum([el**2 for el in B])**(-1/2)
B_amps = C_B*B

def build_A(x):
    cons = []
    for i, x_ij in enumerate(x):
        if x_ij == 1:
            cons.append(connections[i])
    A = np.zeros((4, 4))
    for i in range(4):
        for j in range(4):
            if i == j:
                A[i, j] = 1/R_env + np.sum([1/R_dict[con] if i in con else 0 for con in cons])
            elif i < j:
                if (i, j) in cons:
                    A[i, j] = -1/R_dict[(i, j)]
            elif i > j:
                if (j, i) in cons:
                    A[i, j] = -1/R_dict[(j, i)]
    return A

# Create all possible configurations in natural order
bitstrings = list(itertools.product([0, 1], repeat=len(connections)))

# Calculate the corrsponding costs
costs = []
cond_list = []
min_eigval_list = []
for bitstring in bitstrings:
    A = build_A(bitstring)
    cond_list.append(np.linalg.cond(A))
    min_eigval_list.append(np.min(np.linalg.eigvals(A*C_l**2/2)))
    T = np.linalg.solve(A, B)
    costs.append(T[0]*2*C_B/C_l**2)
    # costs.append(T[0])

In [ ]:
kappa_list = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 32, 38]
epsilon_list = [0.1, 0.01, 0.001]
results_dict = {}

for kappa in kappa_list:
    for epsilon in epsilon_list:
        print(f"kappa={kappa}, epsilon={epsilon}")
        pcoefs, C_p = pyqsp.poly.PolyOneOverX().generate(kappa, return_coef=True, ensure_bounded=True, return_scale=True, epsilon=epsilon)
        phi_pyqsp = QuantumSignalProcessingPhases(pcoefs, signal_operator="Wx")
        degree = len(phi_pyqsp)-1
        print(f"degree={degree}")

        costs_scaled = np.array(costs)*C_p

        q_aux = ["q"]
        l_aux = [f"l({i})" for i in range(int(np.log2(len(conductance_coeffs_amps))))]
        flag_aux = ["flag"]
        l_prime_aux = ["l'"]
        data_wires = [f"d({i})" for i in range(2)]
        
        wires = q_aux + l_aux + flag_aux + l_prime_aux + data_wires
        dev = qml.device("lightning.qubit", wires=wires)
        
        @qml.qnode(dev)
        def qsvt_circuit(config):
            L_demo(B_amps, connections, conductance_coeffs_amps, phi_pyqsp, q_aux[0], config, l_aux, flag_aux[0], l_prime_aux[0], data_wires)
            return qml.state()
        
        import multiprocessing
        from itertools import product
        
        # Generate all configurations
        all_configs = [list(bits) for bits in product([0, 1], repeat=6)]
        
        # Use 'fork' context to avoid multiprocessing issues in Jupyter
        ctx = multiprocessing.get_context("fork")
        
        global process_config
        def process_config(config_index):
            """Runs qsvt_circuit on a given config and returns the amplitude."""
            i, config = config_index
            qsvt_state = qsvt_circuit(config)
            return qsvt_state[0]
        
        # Create a list of indexed configurations
        indexed_configs = [(i, config) for i, config in enumerate(all_configs)]
        
        # Use multiprocessing Pool to parallelize computation
        with ctx.Pool(processes=multiprocessing.cpu_count()) as pool:
            expected_amplitudes = pool.map(process_config, indexed_configs)

        results_dict[(kappa, epsilon)] = [expected_amplitudes, costs_scaled, C_p, degree]


        with open("QSVT_results_dict.pkl", 'wb') as f:
            pickle.dump(results_dict, f)

        print("---")